In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Rectal_Cancer/GSE40492'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Molecular Markers for Predicting Treatment Outcome in Patients with Rectal Cancer: A Comprehensive Analysis from the German Rectal Cancer Trials"
!Series_summary	"Background: Validated markers to predict outcome in rectal cancer patients treated with multimodal therapy remain elusive. Identifying molecular profiles for disease prognosis would be required for the design of clinical trials aimed at optimizing risk-adapted therapies. We have therefore used whole genome expression profiling of tumors of a large cohort of patients enrolled in the multicenter trials of the German Rectal Cancer Study Group (GRCSG) to identify molecular profiles for individualized therapy.  Methods: We prospectively collected pretherapeutic biopsies from patients (n=300) treated according to the GRCSG trial guidelines from seven different German surgical departments using rigid quality controls. These samples were profiled by global gene expression analysis and a classifi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # Gene expression profiling data is mentioned in the series summary

# Identify rows corresponding to variables
trait_row = None  # No explicit mention or single value category found for 'Rectal_Cancer'
age_row = 1  # 'age' values are present
gender_row = 2  # 'Sex' values are present

# Define conversion functions

def convert_trait(value):
    return None  # As trait is not available

def convert_age(value):
    try:
        return float(value.split(": ")[1])
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    gender = value.split(": ")[1].strip().lower()
    if gender == "female":
        return 0
    elif gender == "male":
        return 1
    return None

# Save cohort information
save_cohort_info('GSE40492', './preprocessed/Rectal_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Extract clinical features if available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Rectal_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Rectal_Cancer/trait_data/GSE40492.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
